# Kenyan Real Estate Market Analysis
---
## 1. Environment and Database Connection
Loading environment variables and establishing a connection to the PostgreSQL database.

In [ ]:
import pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

host = os.getenv('AIVEN_DB_HOST')
port = os.getenv('AIVEN_DB_PORT')
name = os.getenv('AIVEN_DB_NAME')
user = os.getenv('AIVEN_DB_USER')
password = os.getenv('AIVEN_DB_PASSWORD')

database_url = f"postgresql://{user}:{password}@{host}:{port}/{name}?sslmode=require"
engine = create_engine(database_url)

query = """
SELECT * FROM cleaned_listings
WHERE price_kes IS NOT NULL
"""

df = pd.read_sql(query, engine)
print(f"Total listings: {len(df)}")

## 2. Statistical Overview
Inspecting the data types and basic descriptive statistics.

In [ ]:
df.info()
df.describe()

## 3. Marketplace and County Analysis
Analyzing where the data is coming from and which counties dominate the listings.

In [ ]:
source_counts = df['source_site'].value_counts()
source_counts.plot(kind='bar', color='steelblue')
plt.title('Listings by Source Website')
plt.show()

county_stats = df.groupby('county').agg({
    'price_kes': ['mean', 'median', 'count']
}).round(2)
county_stats.columns = ['avg_price', 'median_price', 'count']
county_stats = county_stats.sort_values('count', ascending=False).head(10)

county_stats['avg_price'].plot(kind='barh', color='coral')
plt.title('Average Property Price by County (Top 10)')
plt.show()

## 4. Property Types and Price Distributions
Breaking down the types of properties listed and visualizing price density.

In [ ]:
property_types = df['property_type'].value_counts()
plt.pie(property_types.values, labels=property_types.index, autopct='%1.1f%%', startangle=90)
plt.title('Property Type Distribution')
plt.axis('equal')
plt.show()

df['price_kes'].hist(bins=50, color='teal', edgecolor='black')
plt.title('Price Distribution')
plt.show()

## 5. Structural and Neighborhood Analysis
Investigating how bedrooms affect price and diving into Nairobi's specific neighborhoods.

In [ ]:
bedroom_stats = df[df['bedrooms'].notna()].groupby('bedrooms').agg({
    'price_kes': ['mean', 'median', 'count']
}).round(2)
bedroom_stats.columns = ['avg_price', 'median_price', 'count']
bedroom_stats[bedroom_stats.index <= 6]['avg_price'].plot(kind='line', marker='o', color='purple')
plt.title('Average Price by Number of Bedrooms')
plt.show()

nairobi_df = df[df['county'] == 'Nairobi']
neighborhood_stats = nairobi_df.groupby('neighborhood').agg({
    'price_kes': ['mean', 'count']
}).round(2)
neighborhood_stats.columns = ['avg_price', 'count']
neighborhood_stats = neighborhood_stats[neighborhood_stats['count'] >= 5].sort_values('avg_price', ascending=False).head(15)

neighborhood_stats['avg_price'].plot(kind='barh', color='darkgreen')
plt.title('Average Price by Nairobi Neighborhood')
plt.show()

## 6. Nairobi vs Mombasa Comparison
Comparative analysis between the two largest real estate markets in Kenya.

In [ ]:
comparison_df = df[df['county'].isin(['Nairobi', 'Mombasa'])]
comparison_df.boxplot(column='price_kes', by='county')
plt.title('Price Distribution: Nairobi vs Mombasa')
plt.suptitle('')
plt.show()

## 7. Key Findings
Summary of critical market metrics.

In [ ]:
insights = {
    'Total Listings': len(df),
    'Average Price (KES)': f"{df['price_kes'].mean():,.2f}",
    'Median Price (KES)': f"{df['price_kes'].median():,.2f}",
    'Most Common Property Type': df['property_type'].mode()[0],
    'Most Listed County': df['county'].mode()[0],
    'Active Sources': df['source_site'].nunique(),
}
for key, value in insights.items():
    print(f"{key}: {value}")